# MNIST with Tensorflow Serving

Let's train a model first

In [1]:
# !pip install tensorflow==2.3.0
# !pip install keras==2.4.3
# !pip install pydot==1.4.2

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import (Input, Model, layers, metrics, utils, models)

2021-08-15 06:07:10.737621: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-15 06:07:10.737671: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [22]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

11493376/11490434 [==============================] - 0s 0us/step
Number of original training examples: 60000
Number of original test examples: 10000


In [23]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [22]:
def get_model(IMAGE_SHAPE):
    """
    creates and returns a simple CNN model.
    """
    # Define the tensors for the two input images
    input_twin = Input(IMAGE_SHAPE)
    x = layers.Conv2D(512, (2, 2), activation="relu")(input_twin)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256, (2, 2), activation="relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (1, 1), activation="relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(10, name="final_dense", activation='softmax')(x)
    return Model(input_twin, outputs)

In [25]:
INPUT_SHAPE = (28, 28, 1)
model = get_model(INPUT_SHAPE)

In [26]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 512)       2560      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 512)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 256)       524544    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 128)         32896     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 3, 128)         0   

In [27]:
_learning_rate = 1e-3
model.compile(optimizer = tf.keras.optimizers.Adam(_learning_rate), 
              loss = tf.keras.losses.categorical_crossentropy, 
              metrics = ['accuracy'])

In [13]:
model.fit(x_train, y_train, validation_split=0.3, epochs = 5)

Epoch 1/5
1313/1313 [==============================] - 307s 234ms/step - loss: 0.2168 - accuracy: 0.9315 - val_loss: 0.0945 - val_accuracy: 0.9689
Epoch 2/5
1313/1313 [==============================] - 318s 243ms/step - loss: 0.0800 - accuracy: 0.9738 - val_loss: 0.0685 - val_accuracy: 0.9783
Epoch 3/5
1313/1313 [==============================] - 299s 228ms/step - loss: 0.0619 - accuracy: 0.9804 - val_loss: 0.0619 - val_accuracy: 0.9799
Epoch 4/5
1313/1313 [==============================] - 281s 214ms/step - loss: 0.0492 - accuracy: 0.9840 - val_loss: 0.0592 - val_accuracy: 0.9817
Epoch 5/5
1313/1313 [==============================] - 313s 238ms/step - loss: 0.0401 - accuracy: 0.9866 - val_loss: 0.0534 - val_accuracy: 0.9838


In [22]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 20s 62ms/step - loss: 0.0471 - accuracy: 0.9850


[0.04712369292974472, 0.9850000143051147]

In [2]:
model = models.load_model('serving/1')

2021-08-15 06:07:46.049676: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-15 06:07:46.052899: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-15 06:07:46.052921: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-15 06:07:46.052944: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-689f26ff-13bf-4496-9e46-e7b1ce7968de): /proc/driver/nvidia/version does not exist
2021-08-15 06:07:46.053195: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operatio

Creating an export path with the model version

In [40]:
VERSION = 1
BASE_PATH = os.path.join(os.getcwd(),"serving")
EXPORT_PATH = os.path.join(BASE_PATH, str(VERSION))
print(EXPORT_PATH)

/work/tensorflow-serving/serving/1


In [31]:
models.save_model(model, 
                  filepath= EXPORT_PATH, 
                  include_optimizer= True)

INFO:tensorflow:Assets written to: serving/1/assets


Getting the model information

In [4]:
!saved_model_cli show --dir {EXPORT_PATH} --all

2021-08-15 06:08:05.093730: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-15 06:08:05.099758: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_

Installing tensorflow-model-server

In [5]:
SUDO_IF_NEEDED=""

In [6]:
print(f'!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -')

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" |  tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg |  apt-key add -


In [7]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" |  tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg |  apt-key add -

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  35457      0 --:--:-- --:--:-- --:--:-- 35457
OK


In [12]:
!{SUDO_IF_NEEDED} apt update

Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [51.9 kB]
Get:4 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:5 http://security.debian.org/debian-security buster/updates/main amd64 Packages [301 kB]
Get:6 http://deb.debian.org/debian buster/main amd64 Packages [7,907 kB]
Get:7 http://deb.debian.org/debian buster-updates/main amd64 Packages [15.2 kB]
Get:8 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 Packages [340 B]
Get:9 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server-universal amd64 Packages [347 B]
Fetched 8,466 kB in 3s (2,759 kB/s)



8 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [13]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server




The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 326 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.5.1 [326 MB]
Fetched 326 MB in 7s (44.0 MB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 31086 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.5.1_all.deb ...
Unpacking tensorflow-model-server (2.5.1) ...
Setting up tensorflow-model-server (2.5.1) ...


Start running TensorFlow Serving
Now we will load and model and start making REST requests.

- ***rest_api_port***: The port that you'll use.
- ***model_name***: The URL for the requests.
- ***model_base_path***: The path to the saved model.

In [42]:
print(BASE_PATH)

/work/tensorflow-serving/serving


In [30]:
!export EXPORT_PATH='/work/tensorflow-serving/serving'

In [15]:
# RUN THIS IN ANOTHER TERMINAL
!nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=mnist_model \
  --model_base_path="${BASE_PATH}" >server.log 2>&1

^C


In [48]:
!tail server.log

2021-08-15 06:27:21.728784: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /work/tensorflow-serving/serving/1
2021-08-15 06:27:21.733481: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 60671 microseconds.
2021-08-15 06:27:21.734088: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /work/tensorflow-serving/serving/1/assets.extra/tf_serving_warmup_requests
2021-08-15 06:27:21.735558: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: default version: 1}
2021-08-15 06:27:21.736052: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-08-15 06:27:21.736099: I tensorflow_serving/model_servers/server.cc:367] Profiler service is enabled
2021-08-15 06:27:21.737017: I tensorflow_serving/model_servers/server.c

In [44]:
import json
import requests

In [45]:
json_data = {"signature_name": "serving_default", "instances": x_test[0:3].tolist()}

In [46]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/mnist_model:predict', data=json_data, headers=headers)

predictions = json.loads(json_response.text)['predictions']

KeyError: 'predictions'

In [47]:
json_response

<Response [400]>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=689f26ff-13bf-4496-9e46-e7b1ce7968de' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>